In [3]:
# Set Open AI key
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
from langchain.schema import StrOutputParser
from langchain.agents import load_tools, create_react_agent, AgentExecutor
from langchain import hub
from langchain_core.tools import tool

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
# Define the LLM from the Hugging Face model ID
llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 200}
)

prompt = "What is an MRI?"
response = llm.invoke(prompt)
print(response)

: 

In [8]:
template = "Explain this concept simply and concisely: {concept}"
prompt_template = PromptTemplate.from_template(
    template=template
)

prompt = prompt_template.invoke({"concept": "Prompting LLMs"})
print(prompt)

text='Explain this concept simply and concisely: Prompting LLMs'


In [ ]:
# ChatPromptTemplate supports prompting with roles
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a calculator that responds with math."),
        ("human", "What is 2 plus 2?"),
        ("ai", "2+2=4"),
        ("human", "Solve this math problem: {math}")
    ]
)
llm_chain = template | llm
math='What is five times five?'

response = llm_chain.invoke({"math": math})
print(response.content)

In [ ]:
# FewShotPromptTemplate

# Formatting prompt
example_prompt = PromptTemplate.from_template("{q}\n{a}")

prompt = example_prompt.invoke({"q": "What's the capital of Italy?",
                                "a": "Rome"})
print(prompt.text)

# Few-shot prompt
prompt_template = FewShotPromptTemplate(
    examples=examples,                  # List of dicts
    example_prompt=example_prompt,      # Formatted template
    suffix="Question: {input}",         # Format user input
    input_variables=["input"]
)

# Invoke
prompt = prompt_template.invoke({"input": "What is the name of Henry Campbell's dog?"})
print(prompt.text)

# Integrate with chain
llm_chain = prompt_template | llm
response = llm_chain.invoke({"input": "What is the name of Barack Obama's dog?"})
print(response.content)

In [ ]:
# Sequential chains

# Output -> input
destination_prompt = PromptTemplate(
    input_variables=["destination"],
    template="I am planning a trip to {destination}. Can you suggest some activities for me to do there?"
)
activities_prompt = PromptTemplate(
    input_variables=["activities"],
    template="I only have one day, so you can create an itinerary from your top 3 activities: {activities}."
)

seq_chain = ({"activities": destination_prompt | llm | StrOutputParser()}
    | activities_prompt
    | llm
    | StrOutputParser())

print(seq_chain.invoke({"destination": "Costa Rica"}))

## Langchain Agents

In LangChain, agents use language models to determine actions. Agents often use tools, which are functions called by the agent to interact with the system. These tools can be high-level utilities to transform inputs, or they can be task-specific. Agents can even use chains and other agents as tools!

### ReAct agents

ReAct stands for reasoning and acting, and this is exactly how the agent operates. It prompts the model using a repeated loop of thinking, acting, and observing. If we were to ask a ReAct agent that had access to a weather tool, "What is the weather like in Kingston, Jamaica?", it would start by thinking about the task and which tool to call, call that tool using the information, and observe the results from the tool call.

### LangGraph

To implement agents, we'll be using LangGraph, which is branch of the LangChain ecosystem specifically for designing agentic systems, or systems including agents. Like LangChain's core library, it's is built to provide a unified, tool-agnostic syntax. We'll be using the following version in this course.

### Example

We'll create a ReAct agent that can solve math problems - something most LLMs struggle with. We import create_react_agent from langgraph and the load_tools() function. We initialize our LLM, and load the llm-math tool using the load_tools() function. To create the agent, we pass the LLM and tools to create_react_agent(), Just like chains, agents can be executed with the .invoke() method. Here, we pass the chat model a message to find the square root of 101, which isn't a whole number. Let's see how the agent approaches the problem!

There's a lot of metadata in the output, so we've trimmed it for brevity. We can see that executing the agent resulted in a series of messages. The first is our prompt defining the problem; the second is created by the model to identify the tool to use and to convert our query into mathematical format; the third is the result of the tool call, and the final message is the model's response after observing the tool's answer, which it decided to round to two decimal places. If we just want the final response, we can subset the final message and extract it's content with the .content attribute. 

In [4]:
# ReAct agent
tools = load_tools(["llm-math"], llm=llm)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": "What is the square root of 101?"})
print(response["output"])
# print(response['messages'][-1].content)

c:\Users\ash\Desktop\langchain-for-good\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


To find the square root of 101, I will use the Calculator tool to perform the calculation. 
Action: Calculator
Action Input: 101**0.5Answer: 10.04987562112089I now know the final answer.  
Final Answer: The square root of 101 is approximately 10.05.

> Finished chain.
The square root of 101 is approximately 10.05.


In [6]:
# Tool formats
print(tools[0].name)
print(tools[0].description)
print(tools[0].return_direct) # indicates whether the agent should stop after invoking this tool

Calculator
Useful for when you need to answer questions about math.
False


In [6]:
# Define custom function
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income."""
    net_income = revenue - expenses
    
    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

print(financial_report(company_name="LemonadeStand", revenue=100, expenses=50))

Financial Report for LemonadeStand:
Revenue: $100
Expenses: $50
Net Income: $50



In [7]:
# Function -> tool

@tool
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income."""
    net_income = revenue - expenses
    
    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

# Examine
print(financial_report.name)
print(financial_report.description)
print(financial_report.return_direct)
print(financial_report.args)

financial_report
Generate a financial report for a company that calculates net income.
False
{'company_name': {'title': 'Company Name', 'type': 'string'}, 'revenue': {'title': 'Revenue', 'type': 'integer'}, 'expenses': {'title': 'Expenses', 'type': 'integer'}}


In [ ]:
# Integrate
tools = load_tools(["llm-math"], llm=llm)
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, [financial_report], prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

messages = agent_executor.invoke({"input": "TechStack generated made $10 million with $8 million of costs. Generate a financial report."})
print(messages['messages'][-1].content)

In [ ]:
# Define a function to retrieve customer info by-name
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    # Filter customers for the customer's name
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Call the function on Peak Performance Co.
print(retrieve_customer_info(name="Peak Performance Co."))

@tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()

# Create a ReAct agent
agent = create_react_agent(llm, [retrieve_customer_info])

# Invoke the agent on the input
messages = agent.invoke({"messages": [("human", "Create a summary of our customer: Peak Performance Co.")]})
print(messages['messages'][-1].content)

## Integrating document loaders

### Retrieval Augmented Generation (RAG)

Pre-trained language models don't have access to external data sources - their understanding comes purely from their training data. This means that if we require our model to have knowledge that goes beyond its training data, which could be company data or knowledge of more recent world events, we need a way of integrating that data. In RAG, a user query is embedded and used to retrieve the most relevant documents from the database. Then, these documents are added to the model's prompt so that the model has extra context to inform its response.

### RAG development steps

There are three primary steps to RAG development in LangChain. The first is loading the documents into LangChain with document loaders. Next, is splitting the documents into chunks. Chunks are units of information that we can index and process individually. The last step is encoding and storing the chunks for retrieval, which could utilize a vector database if that meets the needs of the use case. We'll discuss all of these steps throughout the next chapter, but for now, we'll start with document loaders.

### LangChain document loaders

LangChain document loaders are classes designed to load and configure documents for integration with AI systems. LangChain provides document loader classes for common file types such as CSV and PDFs. There are also additional loaders provided by 3rd parties for managing unique document formats, including Amazon S3 files, Jupyter notebooks, audio transcripts, and many more. In this video, we will practice loading data from three common formats: PDFs, CSVs, and HTML. LangChain has excellent documentation on all of its document loaders, and there's a lot of overlap in syntax, so explore at your leisure!

    1 https://python.langchain.com/docs/integrations/document_loaders

### PDF document loader

There are a few different types of PDF loaders in LangChain, and there is documentation available online for each. In this video, we'll use the PyPDFLoader. We instantiate the PyPDFLoader class, passing in the path to the PDF file we're loading. Finally, we use the .load() method to load the document into memory, and assign the resulting object to the data variable. We can then check the output to confirm that we have loaded it. Note that this document loader requires installation of the pypdf package as a dependency.

### HTML document loader

Finally, we can load HTML files using the UnstructuredHTMLLoader class. We can access the document's contents, again, with subsetting, and extract the document's metadata with the metadata attribute. 

## Splitting external data for retrieval

### CharacterTextSplitter to split documents

Let's start with CharacterTextSplitter. This method splits based on the separator first, then evaluates chunk_size and chunk_overlap to check if it's satisfied. We call CharacterTextSplitter, passing the separator to split on, along with the chunk_size and chunk_overlap. Applying the splitter to the quote with the .split_text() method, and printing the output, we can see that we have a problem: each of these chunks contains more characters than our specified chunk_size. CharacterTextSplitter splits on the separator in an attempt to make chunks smaller than chunk_size, but in this case, splitting on the separator was unable to return chunks below our chunk_size. Let's take a look at a more robust splitting method!

### RecursiveCharacterTextSplitter

RecursiveCharacterSplitter takes a list of separators to split on, and it works through the list from left to right, splitting the document using each separator in turn, and seeing if these chunks can be combined while remaining under chunk_size. Let's split the quote using the same chunk_size and chunk_overlap.

Notice how the length of each chunk varies. The class split by paragraphs first, and found that the chunk size was too big; likewise for sentences. It got to the third separator: splitting words using the space separator, and found that words could be combined into chunks while remaining under the chunk_size character limit. However, some of these chunks are too small to contain meaningful context, but this recursive implementation may work better on larger documents.

### RecursiveCharacterTextSplitter with HTML

We can also use split other file formats, like HTML. Recall that we can load HTML using UnstructuredHTMLLoader. Defining the splitter is the same, but for splitting documents, we use the .split_documents() method instead of .split_text() to perform the split. 